# [Pyomo.GDP](./index.ipynb) Logical Expression System Demo - Supported expressions

This is a demo file to show off the supported expressions in the logical expression system.
The code currently relies on the logic-v1 branch at https://github.com/qtothec/pyomo/tree/logic-v1.

Currently supported operators include:

1. [Standard Logical Operators](#std.ops)
  - [Conjunction](#logical.and)
  - [Disjunction](#logical.or)
  - [Negation](#logical.not)
  - [Exclusive OR](#logical.xor)
  - [Implication](#logical.implies)
  - [Equivalence](#logical.equiv)
2. [Constraint Programming Operators](#cp.ops)
  - [Exactly N](#logical.exactly)
  - [At Most N](#logical.atmost)
  - [At Least N](#logical.atleast)
3. [Advanced Use](#advanced)

First, we need to do a little setup, importing the pyomo environment and the logical expression system helper functions.\
We also define a function to generate a fresh model for each example that we look at.

In [1]:
from pyomo.environ import *
from pyomo.gdp import *
from pyomo.core.expr.logical_expr import *
from pyomo.core.plugins.transform.logical_to_linear import update_boolean_vars_from_binary

In [2]:
def build_new_model(n=4):
    m = ConcreteModel()
    m.s = RangeSet(n)
    m.Y = BooleanVar(m.s)
    return m

## Standard Logical Operators <a id="std.ops"></a>

### Logical AND <a id="logical.and"></a>


Usage examples:
- <code>Y<sub>1</sub> \& Y<sub>2</sub></code>
- <code>Y<sub>1</sub>.and_(Y<sub>2</sub>)</code>
- <code>And(Y<sub>1</sub>, Y<sub>2</sub>)</code>

In [3]:
m = build_new_model()
m.p = LogicalStatement(expr=m.Y[1] & m.Y[2])
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=2, Index=logic_to_linear_index, Active=True
    Key : Lower : Body          : Upper : Active
      1 :   1.0 : Y_asbinary[1] :   1.0 :   True
      2 :   1.0 : Y_asbinary[2] :   1.0 :   True
p : Size=1, Index=None, Active=False
    Key  : Body        : Active
    None : Y[1] & Y[2] :  False


### Logical OR <a id="logical.or"></a>

Usage examples:
- <code>Y<sub>1</sub> | Y<sub>2</sub></code>
- <code>Y<sub>1</sub>.or_(Y<sub>2</sub>)</code>
- <code>Or(Y<sub>1</sub>, Y<sub>2</sub>)</code>

In [4]:
m = build_new_model()
m.p = LogicalStatement(expr=m.Y[1] | m.Y[2])
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=1, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                          : Upper : Active
      1 :   1.0 : Y_asbinary[1] + Y_asbinary[2] :  +Inf :   True
p : Size=1, Index=None, Active=False
    Key  : Body        : Active
    None : Y[1] | Y[2] :  False


### Logical Negation <a id="logical.not"></a>

Usage examples:
- <code>~Y<sub>1</sub>></code>
- <code>Not(Y<sub>1</sub>)</code>

In [5]:
m = build_new_model()
m.p = LogicalStatement(expr=~m.Y[1])
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=1, Index=logic_to_linear_index, Active=True
    Key : Lower : Body              : Upper : Active
      1 :   1.0 : 1 - Y_asbinary[1] :   1.0 :   True
p : Size=1, Index=None, Active=False
    Key  : Body  : Active
    None : ~Y[1] :  False


### Logical Exclusive OR <a id="logical.xor"></a>

Usage examples:
- <code>Y<sub>1</sub> ^ Y<sub>2</sub></code>
- <code>Y<sub>1</sub>.xor(Y<sub>2</sub>)</code>
- <code>Xor(Y<sub>1</sub>, Y<sub>2</sub>)</code>

In [6]:
m = build_new_model()
m.p = LogicalStatement(expr=m.Y[1] ^ m.Y[2])
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=2, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                  : Upper : Active
      1 :   1.0 :         Y_asbinary[1] + Y_asbinary[2] :  +Inf :   True
      2 :   1.0 : 1 - Y_asbinary[1] + 1 - Y_asbinary[2] :  +Inf :   True
p : Size=1, Index=None, Active=False
    Key  : Body        : Active
    None : Y[1] ^ Y[2] :  False


### Logical Implication <a id="logical.implies"></a>

Usage examples:
- <code>Y<sub>1</sub> >> Y<sub>2</sub></code>
- <code>Y<sub>2</sub> \<\< Y<sub>1</sub></code>
- <code>Y<sub>1</sub>.implies(Y<sub>2</sub>)</code>
- <code>Implies(Y<sub>1</sub>, Y<sub>2</sub>)</code>

In [7]:
m = build_new_model()
m.p = LogicalStatement(expr=m.Y[1] >> m.Y[2])
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=1, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                              : Upper : Active
      1 :   1.0 : 1 - Y_asbinary[1] + Y_asbinary[2] :  +Inf :   True
p : Size=1, Index=None, Active=False
    Key  : Body         : Active
    None : Y[1] >> Y[2] :  False


### Logical Equivalence <a id="logical.equiv"></a>

Usage examples:
- <code>Y<sub>1</sub> == Y<sub>2</sub></code>
- <code>Y<sub>1</sub>.equivalent_to(Y<sub>2</sub>)</code>
- <code>Equivalent(Y<sub>1</sub>, Y<sub>2</sub>)</code>

In [8]:
m = build_new_model()
m.p = LogicalStatement(expr=m.Y[1] == m.Y[2])
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=2, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                              : Upper : Active
      1 :   1.0 : 1 - Y_asbinary[2] + Y_asbinary[1] :  +Inf :   True
      2 :   1.0 : 1 - Y_asbinary[1] + Y_asbinary[2] :  +Inf :   True
p : Size=1, Index=None, Active=False
    Key  : Body          : Active
    None : Y[1] iff Y[2] :  False


## Constraint Programming Inspired Operators <a id="cp.ops"></a>

We provide support for some logical operators inspired by constraint programming, in particular the knapsack-type constraints `Exactly`, `At Most` and `At Least`.

### Exactly N <a id="logical.exactly"></a>

Usage examples:
- <code>Exactly(3, Y<sub>1</sub>, Y<sub>2</sub>, Y<sub>3</sub>, Y<sub>4</sub>)</code>
- <code>Exactly(3, Y)</code>

In [9]:
m = build_new_model()
m.p = LogicalStatement(expr=Exactly(3, m.Y))
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=1, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                                          : Upper : Active
      1 :   3.0 : Y_asbinary[1] + Y_asbinary[2] + Y_asbinary[3] + Y_asbinary[4] :   3.0 :   True
p : Size=1, Index=None, Active=False
    Key  : Body                                 : Active
    None : Exactly(3: [Y[1], Y[2], Y[3], Y[4]]) :  False


### At Most N <a id="logical.atmost"></a>

Usage examples:
- <code>AtMost(3, Y<sub>1</sub>, Y<sub>2</sub>, Y<sub>3</sub>, Y<sub>4</sub>)</code>
- <code>AtMost(3, Y)</code>

In [10]:
m = build_new_model()
m.p = LogicalStatement(expr=AtMost(3, m.Y))
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=1, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                                          : Upper : Active
      1 :  -Inf : Y_asbinary[1] + Y_asbinary[2] + Y_asbinary[3] + Y_asbinary[4] :   3.0 :   True
p : Size=1, Index=None, Active=False
    Key  : Body                                : Active
    None : AtMost(3: [Y[1], Y[2], Y[3], Y[4]]) :  False


### At Least N <a id="logical.atleast"></a>

Usage examples:
- <code>AtLeast(3, Y<sub>1</sub>, Y<sub>2</sub>, Y<sub>3</sub>, Y<sub>4</sub>)</code>
- <code>AtLeast(3, Y)</code>

In [11]:
m = build_new_model()
m.p = LogicalStatement(expr=AtLeast(3, m.Y))
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=1, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                                          : Upper : Active
      1 :   3.0 : Y_asbinary[1] + Y_asbinary[2] + Y_asbinary[3] + Y_asbinary[4] :  +Inf :   True
p : Size=1, Index=None, Active=False
    Key  : Body                                 : Active
    None : AtLeast(3: [Y[1], Y[2], Y[3], Y[4]]) :  False


# Advanced Use <a id="advanced"></a>

Support for complex nested expressions is a key benefit of the logical expression system.
Below are examples of expressions that we support, and with some, an explanation of their implementation.

> **Note:** When using infix operators (e.g. `&` for `and`, `|` for `or`, `>>` for `implies`), exercise caution around order-of-operations.
>
> **Use parentheses.**
> Code that reads `m.Y[1] & m.Y[2] | m.Y[3]` can be ambiguous, because $(Y_1 \wedge Y_2) \vee Y_3$ is different from $Y_1 \wedge (Y_2 \vee Y_3)$.
> `(m.Y[1] & m.Y[2]) | m.Y[3]` is unambiguous.
>
> This is particularly important for implication, since in Python, the bitwise shift operator (`>>`) that we borrow for implication binds more strongly than you may expect.


Composition of standard operators: $$Y_1 \vee Y_2 \implies Y_3 \wedge \neg Y_4 \wedge (Y_5 \vee Y_6)$$

In [12]:
m = build_new_model(n=6)
m.p = LogicalStatement(expr=(m.Y[1] | m.Y[2]) >> (m.Y[3] & ~m.Y[4] & (m.Y[5] | m.Y[6])))
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=6, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                              : Upper : Active
      1 :   1.0 :                 1 - Y_asbinary[1] + Y_asbinary[3] :  +Inf :   True
      2 :   1.0 :                 1 - Y_asbinary[2] + Y_asbinary[3] :  +Inf :   True
      3 :   1.0 :             1 - Y_asbinary[1] + 1 - Y_asbinary[4] :  +Inf :   True
      4 :   1.0 :             1 - Y_asbinary[2] + 1 - Y_asbinary[4] :  +Inf :   True
      5 :   1.0 : Y_asbinary[5] + Y_asbinary[6] + 1 - Y_asbinary[1] :  +Inf :   True
      6 :   1.0 : Y_asbinary[5] + Y_asbinary[6] + 1 - Y_asbinary[2] :  +Inf :   True
p : Size=1, Index=None, Active=False
    Key  : Body                                            : Active
    None : Y[1] | Y[2] >> (Y[3] & (~Y[4]) & (Y[5] | Y[6])) :  False


Expressions within constraint programming-type operators: $$\text{AtLeast}(3, Y_1, Y_2 \vee Y_3, Y_4 \Rightarrow Y_5, Y_6)$$

Here, augmented variables may be automatically added to the model as follows:
$$\text{AtLeast}(3, Y_1, Y_A, Y_B, Y_6)\\
Y_A \Leftrightarrow Y_2 \vee Y_3\\
Y_B \Leftrightarrow (Y_4 \Rightarrow Y_5)
$$

In [13]:
m = build_new_model(n=6)
m.p = LogicalStatement(expr=AtLeast(3, m.Y[1], m.Y[2] | m.Y[3], m.Y[4].implies(m.Y[5]), m.Y[6]))
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=7, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                                                                                                    : Upper : Active
      1 :   3.0 : Y_asbinary[1] + logic_to_linear_augmented_vars_asbinary[1] + logic_to_linear_augmented_vars_asbinary[2] + Y_asbinary[6] :  +Inf :   True
      2 :   1.0 :                                                          1 - Y_asbinary[2] + logic_to_linear_augmented_vars_asbinary[1] :  +Inf :   True
      3 :   1.0 :                                                          1 - Y_asbinary[3] + logic_to_linear_augmented_vars_asbinary[1] :  +Inf :   True
      4 :   1.0 :                                          Y_asbinary[2] + Y_asbinary[3] + 1 - logic_to_linear_augmented_vars_asbinary[1] :  +Inf :   True
      5 :   1.0 :                                                              logic_to_linear_augmented_vars_asbinary[2] + Y_asbinary[4] :  +Inf :   True
   

Nested constraint programming-type operators
$$\text{AtLeast}(2, Y_1, \text{Exactly}(2, Y_2, Y_3, Y_4), Y_5, Y_6)$$

Here, we again need to add augmented variables:
$$\text{AtLeast}(2, Y_1, Y_A, Y_5, Y_6)\\
Y_A \Leftrightarrow \text{Exactly}(2, Y_2, Y_3, Y_4)
$$

However, we also need to further interpret the second statement as a disjunction:
$$\text{AtLeast}(2, Y_1, Y_A, Y_5, Y_6)\\
\left[\begin{gathered}Y_A\\\text{Exactly}(2, Y_2, Y_3, Y_4)\end{gathered}\right]
\vee
\left[\begin{gathered}\neg Y_A\\
\left[\begin{gathered}Y_B\\\text{AtLeast}(3, Y_2, Y_3, Y_4)\end{gathered}\right] \vee \left[\begin{gathered}Y_C\\\text{AtMost}(1, Y_2, Y_3, Y_4)\end{gathered}\right]
\end{gathered}\right]
$$
or equivalently,
$$\text{AtLeast}(2, Y_1, Y_A, Y_5, Y_6)\\
\text{Exactly}(1, Y_A, Y_B, Y_C)\\
\left[\begin{gathered}Y_A\\\text{Exactly}(2, Y_2, Y_3, Y_4)\end{gathered}\right]
\vee
\left[\begin{gathered}Y_B\\\text{AtLeast}(3, Y_2, Y_3, Y_4)\end{gathered}\right] \vee \left[\begin{gathered}Y_C\\\text{AtMost}(1, Y_2, Y_3, Y_4)\end{gathered}\right]
$$

In our transformation, we automatically convert these special disjunctions to linear form using a Big M reformulation.

In [14]:
m = build_new_model(n=6)
m.p = LogicalStatement(expr=AtLeast(2, m.Y[1], Exactly(2, m.Y[2], m.Y[3], m.Y[4]), m.Y[5], m.Y[6]))
TransformationFactory('core.logical_to_linear').apply_to(m)
m.logic_to_linear.pprint()  # auto-generated by transformation
m.p.pprint()

logic_to_linear : Size=6, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                                                                                                                                 : Upper : Active
      1 :   2.0 :                                           Y_asbinary[1] + logic_to_linear_augmented_vars_asbinary[1] + Y_asbinary[5] + Y_asbinary[6] :  +Inf :   True
      2 :  -Inf :                                 Y_asbinary[2] + Y_asbinary[3] + Y_asbinary[4] - (1 - logic_to_linear_augmented_vars_asbinary[1] + 2) :   0.0 :   True
      3 :  -Inf :                             2 - 2*(1 - logic_to_linear_augmented_vars_asbinary[1]) - (Y_asbinary[2] + Y_asbinary[3] + Y_asbinary[4]) :   0.0 :   True
      4 :   1.0 : logic_to_linear_augmented_vars_asbinary[1] + logic_to_linear_augmented_vars_asbinary[2] + logic_to_linear_augmented_vars_asbinary[3] :  +Inf :   True
      5 :  -Inf :                             Y_asbinary[2] + Y_asbinary[3] + Y_asbinary[4] -